# MOVEIT!    LAUNCH IT!
#OSCII CODED

## Setup

必要なライブラリをインポートして、初期化を行います。

Import the required libraries and initialize them:

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ros_numpy
import rospy
import tf
from gazebo_ros import gazebo_interface
from sensor_msgs.msg import LaserScan, PointCloud2
from geometry_msgs.msg import Pose, Quaternion
import sys
import time

from utils_notebooks import *
import cv2
import os




In [4]:
import moveit_commander
import moveit_msgs.msg
head = moveit_commander.MoveGroupCommander('head')

In [5]:


## THIS PATH MUST BE SET BY USER () 


#_path_model = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models"   Tenshi Version

_path_model = "/home/oscar/Codes/catkin_mio_ws/src/tmc_wrs_gazebo_world/models"     #Catkin mio version 
#_path_model = "/home/roboworks/Codes/catkin_mio/src/tmc_wrs_gazebo_world/models"     #Catkin mio version 


objs=os.listdir(_path_model)
objs.sort()
print (objs)

['person_standing', 'training_floor', 'training_wall', 'trofast', 'wrc_bin_black', 'wrc_bin_green', 'wrc_bookshelf', 'wrc_container_a', 'wrc_container_b', 'wrc_frame', 'wrc_ground_plane', 'wrc_long_table', 'wrc_stair_like_drawer', 'wrc_tall_table', 'wrc_tray', 'ycb_002_master_chef_can', 'ycb_003_cracker_box', 'ycb_004_sugar_box', 'ycb_005_tomato_soup_can', 'ycb_006_mustard_bottle', 'ycb_007_tuna_fish_can', 'ycb_008_pudding_box', 'ycb_009_gelatin_box', 'ycb_010_potted_meat_can', 'ycb_011_banana', 'ycb_012_strawberry', 'ycb_013_apple', 'ycb_014_lemon', 'ycb_015_peach', 'ycb_016_pear', 'ycb_017_orange', 'ycb_018_plum', 'ycb_019_pitcher_base', 'ycb_021_bleach_cleanser', 'ycb_022_windex_bottle', 'ycb_024_bowl', 'ycb_025_mug', 'ycb_026_sponge', 'ycb_027_skillet', 'ycb_028_skillet_lid', 'ycb_029_plate', 'ycb_030_fork', 'ycb_031_spoon', 'ycb_032_knife', 'ycb_033_spatula', 'ycb_035_power_drill', 'ycb_036_wood_block', 'ycb_037_scissors', 'ycb_038_padlock', 'ycb_040_large_marker', 'ycb_042_adjust

In [6]:
#AGAIN THIS PATH MUST BE SET ( IT IS LOCATED IN NOTEBOOK UTILS FILE utils_notebooks.py)

rvizを起動します．ロボットモデル、カメラ映像、ポイントクラウドが表示されています。

In [7]:
#Initialize  Ros Node
rospy.init_node("recognition")


In [8]:
#create a RGBD() object to read xtion
rgbd = RGBD()

In [9]:
#Create a tf listener to read tf tree ( shockingly)
listener = tf.TransformListener()
rospy.sleep(.5)

Launch RViz. The robot model, camera image, and point cloud are displayed:

In [10]:
def gazebo_2_world(x,y):

    x_world= x+2.1
    y_world= -(y-1.2)
    return (x_world,y_world)

def world_2_gazebo(y_world , x_world):

    x= ( x_world - 2.1)
    y= (-y_world + 1.2) 
    return (x , y)

In [11]:
#Get pose of the xtion w.r.t map frame 
#( odom , or hsrb base vfootprint might be other 
#useful reference frames)


trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0))
#trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_link', rospy.Time(0))
euler=tf.transformations.euler_from_quaternion(rot)

trans, euler

([0.6951207926186899, 0.9416752574493436, 1.0040300819993895],
 (-4.896525256896483e-12, 0.5013730988868775, 0.13616287269908978))

In [12]:
#moveit
#MOVE IT MUST BE RUNNING. DECLARE A HEAD POSE
head_val=head.get_current_joint_values()
head_val[0]=np.deg2rad(-80)
head_val[1]=np.deg2rad(20)
#WATCH OUT FOR JOINTS LIMITS (exorcist joke)

In [13]:
#plan and execute target pose
head.set_joint_value_target(head_val)
head.go()

True

In [14]:
#Get pose of the xtion w.r.t map frame 

trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0))
#trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_link', rospy.Time(0))
euler=tf.transformations.euler_from_quaternion(rot)

trans, euler


([0.6248782447294475, 1.042635464118673, 0.9513388291058679],
 (-4.896530150006963e-12, -0.3529383175890137, -1.2525914427054403))

In [16]:
model_name = objs[33]
print model_name

ycb_021_bleach_cleanser


In [17]:
#USE TRANSFORM TREE TF TO SPAWN A STATIC MODEL
# D_to_cam  METERS AWAY FROM XTION ON THE dIRECTION OF IT FOCAL POINT

## PLEASE CHECK YOU HAVE CORRECTED PATH IN utils_notebooks file 


### MUST POINT TO CATKIN_TUTORIALS
### Catkin_mio
### Or wherever your gazebo wrs World and YCB objects models  are stored

###OLD REPOSITORIES (SUBMODULED ONES ARE NO LONGER MANTAINED:)



#x_gaz,y_gaz=world_2_gazebo(1.15,0.02)
deg=0
D_to_cam= 0.6
x_gaz,y_gaz=world_2_gazebo(+trans[0]+ D_to_cam * np.cos(euler[2]), +  trans[1]+ D_to_cam * np.sin(euler[2]))
z_gaz= trans[2]- D_to_cam*np.sin(euler[1])
spawn_object(model_name, model_name, x_gaz,  y_gaz,  z_gaz, 1.5* np.pi , 0,euler[2])
spawn_object(model_name+'1', model_name, x_gaz,  y_gaz, z_gaz ,-.5*np.pi , .5*np.pi, euler[2])     #spawn_object(model_name+'1', model_name, x_gaz,  y_gaz, z_gaz,0.5*np.pi , 0 ,euler[2])
spawn_object(model_name+'2', model_name, x_gaz,  y_gaz,  z_gaz, np.pi,0 ,euler[2])
spawn_object(model_name+'3', model_name, x_gaz,  y_gaz,  z_gaz, 1.5*np.pi ,0,euler[2]+np.pi)

[INFO] [1635863320.602914, 282.489000]: Spawn: ycb_021_bleach_cleanser
[INFO] [1635863320.608401, 282.489000]: Spawn: [ 0.41446892  0.41446892  0.57290097 -0.57290097]
[INFO] [1635863320.617792, 282.489000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1635863320.647741, 282.489000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1635863321.264204, 282.651000]: Spawn status: SpawnModel: Successfully spawned entity
[INFO] [1635863321.266019, 282.651000]: Spawn: ycb_021_bleach_cleanser
[INFO] [1635863321.267557, 282.651000]: Spawn: [ 0.11202838 -0.69817594 -0.11202838  0.69817594]
[INFO] [1635863321.271438, 282.651000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1635863321.295251, 282.657000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1635863321.515595, 282.708000]: Spawn status: SpawnModel: Successfully spawned entity
[INFO] [1635863321.517106, 282.708000]: Spawn: ycb_021_bleach_cleanser
[INFO] [1635863321.518366, 282.708000]: Spawn: [ 5.86147564e-01 -3.58911869e-

In [ ]:
#GET AND SHOW IMAGE
cv2_img = rgbd.get_image()
plt.imshow(cv2_img)
delete_object(model_name)


In [ ]:
delete_object(model_name)
delete_object(model_name+str(1))
delete_object(model_name+str(2))
delete_object(model_name+str(3))


In [ ]:
delete_object(model_name)


In [ ]:
import datetime
ct = datetime.datetime.now()
print("current time:-", ct)

レゴブロックが頭部カメラの画角に入るように頭を少し下げます。rviz上の頭部カメラの映像が変化することを確認しましょう。

Lower the head slightly so that the Lego block is within the viewing angle of the head camera. Check that the image in the head camera in RViz changes accordingly:

In [ ]:
#Capture a frame each degdree while changing static object pose
#I dont love this way of spawining and  deleting, 
#CONTROL VIA ROS TOPIC POSE !
samples_rev=100
start = time.time()
model_name = objs[82]
D_to_cam=0.75

for j in range (3):
    deg=0
    
    for i in range (samples_rev):
        trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0))
        euler=tf.transformations.euler_from_quaternion(rot)
        deg+= np.deg2rad(i*2*np.pi/samples_rev)
        print(i,deg)
        x_gaz,y_gaz=world_2_gazebo(       trans[0]+ D_to_cam * np.cos(euler[2]),   trans[1]+ D_to_cam * np.sin(euler[2])    )
        z_gaz= trans[2]- D_to_cam*np.sin(euler[1])

        #spawn_object("lego2", model_name, x_gaz,  y_gaz, trans[2], 0.8)
        #spawn_object(model_name, model_name, x_gaz,  y_gaz, trans[2]- D_to_cam* np.sin(euler[1]) , 2*np.pi*np.random.rand() ,2*np.pi*np.random.rand(),2*np.pi*np.random.rand())
        spawn_object(model_name, model_name, x_gaz,  y_gaz,  z_gaz, deg ,deg ,euler[2]+.4+.1*j*np.pi)
        
        cap_cnt= i
        rospy.sleep(.1)
        cv2_img = rgbd.get_image()

        cap_name = "/home/roboworks/Desktop/imagesfromlistener/"+model_name+"_"+str(j*30)+"deg""_{}.png".format(cap_cnt)   ##SAVE YOUR IMAGES WHEREVER YOU WAnt
        #cap_name = "imagesfromlistener/image_from_listener_{}.png".format(cap_cnt)
        im_rgb = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(cap_name, im_rgb)
        delete_object(model_name)
end = time.time()


    

In [ ]:
print( 'time elapsed in trainSing video',(end - start)/60,'mins')

ct = datetime.datetime.now()
print("current time:-", ct)

In [ ]:
#Capture a frame each degdree while changing static object pose
#I dont love this way of spawining and  deleting, 
#CONTROL VIA ROS TOPIC POSE !
samples_rev=100
start = time.time()
model_name = objs[83]
D_to_cam=0.75

for j in range (3):
    deg=0
    
    for i in range (samples_rev):
        trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0))
        euler=tf.transformations.euler_from_quaternion(rot)
        deg+= np.deg2rad(i*2*np.pi/samples_rev)
        print(i,deg)
        x_gaz,y_gaz=world_2_gazebo(       trans[0]+ D_to_cam * np.cos(euler[2]),   trans[1]+ D_to_cam * np.sin(euler[2])    )
        z_gaz= trans[2]- D_to_cam*np.sin(euler[1])

        #spawn_object("lego2", model_name, x_gaz,  y_gaz, trans[2], 0.8)
        #spawn_object(model_name, model_name, x_gaz,  y_gaz, trans[2]- D_to_cam* np.sin(euler[1]) , 2*np.pi*np.random.rand() ,2*np.pi*np.random.rand(),2*np.pi*np.random.rand())
        spawn_object(model_name, model_name, x_gaz,  y_gaz,  z_gaz, deg ,deg ,euler[2]+.4+.1*j*np.pi)
        
        cap_cnt= i
        rospy.sleep(.1)
        cv2_img = rgbd.get_image()

        cap_name = "/home/roboworks/Desktop/images2/"+model_name+"_"+str(j*30)+"deg""_{}.png".format(cap_cnt)   ##SAVE YOUR IMAGES WHEREVER YOU WAnt
        #cap_name = "imagesfromlistener/image_from_listener_{}.png".format(cap_cnt)
        im_rgb = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(cap_name, im_rgb)
        delete_object(model_name)
end = time.time()


    

In [ ]:
print( 'time elapsed in training video',(end - start)/60,'mins')
import datetime
ct = datetime.datetime.now()
print("current time:-", ct)

In [ ]:
#Capture a frame each degdree while changing static object pose
#I dont love this way of spawining and  deleting, 
#CONTROL VIA ROS TOPIC POSE !
samples_rev=100
start = time.time()
model_name=objs[81]
D_to_cam=.75
for j in range (3):
    deg=0
    
    for i in range (samples_rev):
        trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0))
        euler=tf.transformations.euler_from_quaternion(rot)
        deg+= np.deg2rad(i*2*np.pi/samples_rev)
        print(i,deg)
        x_gaz,y_gaz=world_2_gazebo(       trans[0]+ D_to_cam * np.cos(euler[2]),   trans[1]+ D_to_cam * np.sin(euler[2])    )
        z_gaz= trans[2]- D_to_cam*np.sin(euler[1])

        #spawn_object("lego2", model_name, x_gaz,  y_gaz, trans[2], 0.8)
        #spawn_object(model_name, model_name, x_gaz,  y_gaz, trans[2]- D_to_cam* np.sin(euler[1]) , 2*np.pi*np.random.rand() ,2*np.pi*np.random.rand(),2*np.pi*np.random.rand())
        spawn_object(model_name, model_name, x_gaz,  y_gaz,  z_gaz, +.4+.1*j*np.pi, 0 ,deg)
        
        cap_cnt= i
        rospy.sleep(.1)
        cv2_img = rgbd.get_image()

        cap_name = "/home/roboworks/Desktop/images3/"+model_name+"_"+str(j*30)+"deg""_{}.png".format(cap_cnt)   ##SAVE YOUR IMAGES WHEREVER YOU WAnt
        #cap_name = "imagesfromlistener/image_from_listener_{}.png".format(cap_cnt)
        im_rgb = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(cap_name, im_rgb)
        delete_object(model_name)
end = time.time()


    

In [ ]:
print( 'time elapsed in trainSing video',(end - start)/60,'mins')


#TO MAKE MR EDD HAPPY MAKE A VIDEO OUT OF THIS FRAMES
#on terminal go to folder containing the images and run
# $ffmpeg -framerate 30  -pattern_type glob -i '*.png' -r 5 video.mp4
# It is advisable to name each video other than video.mp4

In [ ]:
x_gaz,y_gaz=2.66,-1.0

spawn_object("lego1", model_name, x_gaz,  y_gaz, 0.81, 0)
spawn_object("lego2", model_name, x_gaz,  y_gaz+.20, 0.81, 0)


In [ ]:
delete_object("lego1")
delete_object("lego2")
delete_object("lego3")

In [ ]:
low= np.array([0,100,0])
high= np.array([100,200,100])
mask = cv2.inRange(cv2_img, low, high)
mask_buena= cv2.morphologyEx(mask,cv2.MORPH_ERODE,np.ones((9,9),np.uint8))
masked_image = np.copy(img)
masked_image[mask_buena != 0] = [0, 0, 0]

#
### FIND OBJECTS ( CONTOURS FINDING IN MASKED IMAGE)

ret,thresh = cv2.threshold(mask_buena,127,255,0)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
i=find_2nd_biggest_contour_ix(contours)
boundRect = cv2.boundingRect(contours[find_2nd_biggest_contour_ix(contours)])#contours_poly[i])
cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+boundRect[2], boundRect[1]+boundRect[3]), color, 2)
x,y,w,h =boundRect
print (w,h)
if (w < h):


    x=(int)(x-.5*(h-w))
    w=h
    boundRect=x,y,w,h
    cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+boundRect[2], boundRect[1]+boundRect[3]), color2, 2)
if (h < w):


    y=(int)(x-.5*(w-h))
    h=w
    boundRect=x,y,w,h
    cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+boundRect[2], boundRect[1]+boundRect[3]), color2, 2)

img_roi=img[y:y+h,x:x+w,:]
#img_resized=cv2.resize(img_roi,(img_width,img_height))



cv2.imshow('frame', img)
if cv2.waitKey(1) == ord('q'):
    break

In [ ]:
#USE TRANSFORM TREE TF TO SPAWN A STATIC MODEL
# D_to_cam  METERS AWAY FROM XTION ON THE dIRECTION OF IT FOCAL POINT


#x_gaz,y_gaz=world_2_gazebo(1.15,0.02)
deg=0
model_name= 'ycb_065-g_cups'   #for cups
D_to_cam= 0.35
x_gaz,y_gaz=world_2_gazebo(+trans[0]+ D_to_cam * np.cos(euler[2]), -.05+  trans[1]+ D_to_cam * np.sin(euler[2]))
z_gaz= -.03+trans[2]- D_to_cam*np.sin(euler[1])
spawn_object(model_name, model_name, x_gaz,  y_gaz,  z_gaz, 0 ,0 ,euler[2]+.6*np.pi)
spawn_object(model_name+'1', model_name, x_gaz,  y_gaz, z_gaz, 0.5*np.pi,0 ,euler[2]+.8*np.pi)
spawn_object(model_name+'2', model_name, x_gaz,  y_gaz,  z_gaz, np.pi,0 ,euler[2]+np.pi)
spawn_object(model_name+'3', model_name, x_gaz,  y_gaz,  z_gaz, 1.5*np.pi ,0,euler[2]+1.2*np.pi)